# Budget Analysis

---

## Environment Setup

In [24]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [4]:
# Set environment variables from the .env file
load_dotenv()

True

In [5]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [53]:
# Create the client object
from plaid import Client

client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, environment='sandbox')


In [54]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [55]:
INSTITUTION_ID = "ins_109508"

# Create the public token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions', 'income', 'assets'])

# Print the public token
print(create_tkn_response)

{'public_token': 'public-sandbox-d08da9dc-7a0c-4b15-a4d6-ce5f570a87ab', 'request_id': '6mKqdtCqgxKxfDp'}


### 3. Exchange the public token for an access token

In [56]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Print the access toke

print(exchange_response)

{'access_token': 'access-sandbox-6409d8b7-f8f0-46f2-9b01-eb5f326c9138', 'item_id': 'PnMvZgn93yhegjJDK8xnHlxwD61K9pF7zM8oG', 'request_id': 'qpkHeOt23Mg4mOV'}


In [57]:
# Store the access token in a Python variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [67]:
# Retrieve accounts
client.Accounts.get(access_token)

# Print accounts

{'accounts': [{'account_id': 'BezEKae1GquAaXKPm5V4HBP5NnNQWwFw5gdn8',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': '3le4X6lzjWsGxpmkKW78I35pwZwRydsqbEk3R',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'xb31Wpb94dHxekj4aAdmTMwoeaevpQFnPexve',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [68]:
# Set the start and end date 90 days from today

start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now()) 

# Retrieve the transactions for the last 90 days

transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print the transactions

for transactions in transaction_response['transactions']:
    print(json.dumps(transaction_response['transactions'][:2], indent=4, sort_keys=True))



[
    {
        "account_id": "BezEKae1GquAaXKPm5V4HBP5NnNQWwFw5gdn8",
        "account_owner": null,
        "amount": 6.33,
        "authorized_date": null,
        "category": [
            "Travel",
            "Taxi"
        ],
        "category_id": "22016000",
        "date": "2020-07-27",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "merchant_name": "Uber",
        "name": "Uber 072515 SF**POOL**",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },

### 2. Print the categories for each transaction

In [71]:
# Create a for-loop to print the categories for each transaction

transactions = transaction_response['transactions']
for x in transactions:
    category = x['category']
    print(category)


['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']
['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']
['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel'

### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [176]:
# Define an empty DataFrame and set the columns as date, name, amount, and category

empty_df = pd.DataFrame(columns = ['Date', 'Name', 'Amount', 'Category'])

# Retrieve all the transactions

transactions = transaction_response['transactions']
dates = []
name = []
amount = []
categories = []

# Populate the transactions DataFrame with the transactions data

for x in transactions:
    dates.append(x['date'])
    name.append(x['name'])
    amount.append(x['amount'])
    category = x['category'][0]
    categories.append(category)
    
data = [dates, name, amount, categories]
data_df = pd.DataFrame(data)
transactions_df = data_df.transpose()
transactions_df.columns = ['Date', 'Name', 'Amount', 'Category']
transactions_df.head(10)

,Date,Name,Amount,Category
0,2020-07-27,Uber 072515 SF**POOL**,6.33,Travel
1,2020-07-24,Tectra Inc,500,Food and Drink
2,2020-07-23,AUTOMATIC PAYMENT - THANK,2078.5,Payment
3,2020-07-23,KFC,500,Food and Drink
4,2020-07-23,Madison Bicycle Shop,500,Shops
5,2020-07-14,CREDIT CARD 3333 PAYMENT *//,25,Payment
6,2020-07-14,Uber 063015 SF**POOL**,5.4,Travel
7,2020-07-13,ACH Electronic CreditGUSTO PAY 123456,5850,Transfer
8,2020-07-13,CD DEPOSIT .INITIAL.,1000,Transfer
9,2020-07-12,Touchstone Climbing,78.5,Recreation


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [177]:
# Display data types

#transaction_df['Amount'].astype('float')
transactions_df_2['Date'] = pd.to_datetime(transactions_df['Date'])
transaction_df_2.set_index('Date',inplace=True)
print(transaction_df_2.dtypes)

NameError: name 'transactions_df_2' is not defined

In [ ]:
# Set the date column as index

# Display sample data


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [ ]:
# Fetch the income data

# Print the income data


In [ ]:
# Determine the previous year's gross income and print the results


In [ ]:
# Determine the current monthly income and print the results


In [ ]:
# Determine the projected yearly income and print the results


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# Compute the total expenses per category

# Display the expenses per catefory


In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
